In [ ]:
"""uncertainty_principle.ipynb"""
# Cell 1

# Cell 1 script from @dbiersach:
# https://github.com/dbiersach/qis101/blob/0aa4537a02ac7f71d62ce6bbaf54e1e3175a5d51/labs/Session%2013%20-%20Fourier%20Analysis/uncertainty_principle.ipynb

# Cell 2 contains written answers to posed questions about the uncertainty principle in quantum mechanics.

from __future__ import annotations

import typing

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation

if typing.TYPE_CHECKING:
    from typing import Generator

    from matplotlib.axes import Axes
    from matplotlib.lines import Line2D
    from numpy.typing import NDArray

%matplotlib widget

wave_pdf: Line2D
wave_ps: Line2D

sigma: float
ts: NDArray[np.float_]


def f(x: NDArray[np.float_]) -> NDArray[np.float_]:
    # The Gaussian standard normal probability density function
    return (
        1
        / (sigma * np.sqrt(2 * np.pi))
        * np.exp((-1 / 2) * np.power(x, 2) / (np.power(sigma, 2)))
    )


def plot_samples(ts: NDArray[np.float_], ys: NDArray[np.float_], ax: Axes) -> None:
    global wave_pdf
    (wave_pdf,) = ax.plot(ts, ys)
    ax.set_title(f"Particle Location (Probability Density)")
    ax.set_xlabel("Location", loc="right")
    ax.set_ylabel("Probability Density")
    ax.set_ylim(0, 25)


def plot_powerspec(ps: NDArray[np.float_], ax: Axes) -> None:
    global wave_ps
    (wave_ps,) = ax.plot(range(len(ps)), ps, color="green")
    ax.set_title(f"Particle Frequencies")
    ax.set_xlabel("Frequency", loc="right")
    ax.set_ylabel(r"$\Vert Amplitude \Vert$")
    ax.set_ylim(0, 1)


def anim_frame_counter() -> Generator[int, None, None]:
    global sigma
    n = 1
    while n < 1200:
        if n <= 600:
            sigma = 10 / n
        else:
            sigma = 10 / (1200 - n)
        n += 5
        yield n


def anim_draw_frame(t: float) -> tuple[Line2D, Line2D]:
    global sigma
    ys: NDArray[np.float_] = f(ts)
    wave_pdf.set_data(ts, ys)

    sigma /= 30
    ys = f(ts)
    ca: NDArray[np.complex_] = np.fft.rfft(ys) / 2
    ps: NDArray[np.float_] = np.abs(ca) / len(ca)

    wave_ps.set_data(range(len(ps)), ps)

    return (
        wave_pdf,
        wave_ps,
    )


def main() -> None:
    global ts, sigma, anim

    ts = np.linspace(-1, 1, 1000, endpoint=False)

    sigma = 1
    ys: NDArray[np.float_] = f(ts)

    ca: NDArray[np.complex_] = np.fft.rfft(ys) / 2
    ps: NDArray[np.float_] = np.abs(ca) / len(ca)

    plt.close("all")
    plt.figure("Uncertainty Principle", constrained_layout=True, figsize=(10, 4))

    ax_pdf: Axes = plt.subplot(121)
    ax_ps: Axes = plt.subplot(122)

    plot_samples(ts, ys, ax_pdf)
    plot_powerspec(ps, ax_ps)

    anim = FuncAnimation(
        ax_pdf.figure,
        anim_draw_frame,
        anim_frame_counter,
        interval=25,
        blit=True,
        repeat=False,
        cache_frame_data=False,
    )

    plt.show()


main()

In [ ]:
# Cell 2: Questions for Task 13-02

"""
1) Due to the uncertainty principle, the more you know about an object's position, the less you know about its momentum. This
is analagous to hearing a sound wave. The shorter amount of time for which you hear the wave, the less you can tell about
its frequency. Applied to the particle position and its momentum, the amount of knowledge we have about the particle's position
corresponds to the amount of time for which we play a sound wave. The less time we play the wave/the more certain we are of 
the object's position, the less we can tell about the sound wave's frequency/the object's momentum. Put more simply, the more
precision we have in the time/location domain, the less we have in the frequency/momentum domain. The Fourier Transform 
frequencies spread out when the particle's position is confined in reflection of the trade-off in precision.

2) The probability of finding the particle at any point x spreads out infinitely when the power spectrum shows only one
frequency because due to the uncertainty principle, in order for the frequency to be determined with total accuracy and no
error, there would have to be no information known about the particle's position. Therefore, the probability of finding the
particle would have to be the same at any position from -infinity to +infinity, so the PDF spreads out until it appears 
as a flat line on the x-axis.

3) Mathematically, yes, I believe a particle with 100% known momentum could exist anywhere in the universe because knowing
the momentum with 100% certainty implies the particle's position is known with 0% certainty, so a PDF of the particle's 
location would have to display an equal likelihood of finding the particle anywhere in the universe. In reality, we can 
never know something with infinite precision and 100% accuracy, so this scenario would never happen.

Resources:
https://www.3blue1brown.com/lessons/uncertainty-principle
"""
